In [ ]:
import matplotlib.pyplot as plt
import h5py
import numpy as np
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
from scipy import stats as sps

import pytensor.tensor as ptt

import icomo

plt.rcParams.update({'font.size': 8})

In [ ]:
def bootstrap_confidence_interval(data, n=1000, func=np.mean, alpha=0.05):
    resamples = np.random.choice(data.dropna(), size=(n, len(data.dropna())), replace=True)
    perc = np.percentile([func(r) for r in resamples], [100*alpha/2, 100*(1-alpha/2)])
    return [func(data.dropna()) - perc[0], perc[1] - func(data.dropna())]

def percentile_confidence_interval(data, alpha=0.05):
    perc = np.percentile(data.dropna(), [100*alpha/2, 100*(1-alpha/2)])
    return [data.mean() - perc[0], perc[1] - data.mean()]

# System definition

In [ ]:
# standard parameters
_RATE_RECOVER = 1.0 / 3.0
_RATE_LOAD = 1.0 / 0.5
_RATE_DOCK = 1.0 / 1.0
_RATE_PRIME = 1.0 / 0.1
_MAX_RECOVERY = 38.0
_MAX_LOADED = 38.0
_MAX_DOCKED = 20.0
_MAX_PRIMED = 4.0
_MAX_FUSED = 100.0
_RELEASE_PROBABILITY = 0.2

# Define the differential equations
def equations(t, y, args):
    t_args, const_args = args

    RATE_RECOVER = const_args["RATE_RECOVER"]
    RATE_LOAD = const_args["RATE_LOAD"]
    RATE_DOCK = const_args["RATE_DOCK"]
    RATE_PRIME = const_args["RATE_PRIME"]
    MAX_RECOVERY = const_args["MAX_RECOVERY"]
    MAX_LOADED = const_args["MAX_LOADED"]
    MAX_DOCKED = const_args["MAX_DOCKED"]
    MAX_PRIMED = const_args["MAX_PRIMED"]
    RELEASE_PROBABILITY = const_args["RELEASE_PROBABILITY"]

    stim_rate = t_args

    recovery = y["recovery"]
    loaded = y["loaded"]
    docked = y["docked"]
    primed = y["primed"]
    fused = y["fused"]

    # recovery
    fused_to_recovery = RATE_RECOVER * (fused > 0) * (recovery <= MAX_RECOVERY)
    recovery_to_loaded = 4 * RATE_LOAD * (recovery / MAX_RECOVERY) * (1 - loaded / MAX_LOADED)
    loaded_to_docked = 4 * RATE_DOCK * (loaded / MAX_LOADED) * (1 - docked / MAX_DOCKED)
    docked_to_primed = 2 * RATE_PRIME * (docked / MAX_DOCKED) * (1 - primed / MAX_PRIMED)

    drecovery = fused_to_recovery - recovery_to_loaded
    dloaded = recovery_to_loaded - loaded_to_docked
    ddocked = loaded_to_docked - docked_to_primed
    dprimed = docked_to_primed
    dfused = - fused_to_recovery

    # release
    release = RELEASE_PROBABILITY * primed * stim_rate(t) 
    dprimed -= release
    dfused += release

    dy = {
        "recovery": drecovery,
        "loaded": dloaded,
        "docked": ddocked,
        "primed": dprimed,
        "fused": dfused,
    }
    
    return dy

# Fit to experiment

In [ ]:
# get experimental data from xls
excel_file = "exp1_data.xlsx" # change path to the location of your file
df1 = pd.read_excel(excel_file)
# print(df1)

# load exhaustion experiment data from xls
excel_file = "exp2_data.xlsx" 
df2 = pd.read_excel(excel_file)
data2 = df2[df2.columns[1:]].values.T
time = df2["seconds"].values
print(data2.shape)
print(time.shape)
experimental_fusion_rate = np.diff(data2) / np.diff(time)
experimental_fusion_rate = np.concatenate([experimental_fusion_rate, np.zeros((experimental_fusion_rate.shape[0],1))], axis=1)
print(experimental_fusion_rate.shape)

# stim starts at 32.35 s
stim_start = 32.35
inds = time > stim_start
time = time[inds] - time[inds][0]
data2 = data2[:,inds]
# start at 0 not 1
data2 = data2 - data2[:,[0]]
experimental_fusion_rate = experimental_fusion_rate[:,inds]
print(data2.shape)
print(time.shape)

data2_means = np.mean(data2, axis=0)
data2_errs = np.std(data2, axis=0) / np.sqrt(data2.shape[0])

In [ ]:
pps = 10 # points per second

def get_integrator_object(depletiontime, pausetime, testtime):
    len_sim = depletiontime + pausetime + testtime # s
    num_points = int(len_sim * pps)

    t_solve_ODE = np.linspace(0, len_sim, num_points) # timepoints at which the ODE is solved
    t_stim = t_solve_ODE # timepoints at which the stimulus is defined

    # only return the output during the test time
    t_out_inds = np.logical_and((t_solve_ODE > depletiontime + pausetime), (t_solve_ODE < depletiontime + pausetime + testtime))
    t_out = t_solve_ODE[t_out_inds]

    integrator_object = icomo.ODEIntegrator(
        ts_out=t_out,
        t_0=min(t_solve_ODE),
        ts_solver=t_solve_ODE,
        ts_arg=t_stim,
        max_steps=len(t_solve_ODE),
    )
    return integrator_object, t_stim

def get_integrator_object2():

    len_sim = 70 # s
    pps = 17 # points per second later scaled to 1.7 by taking mod 20
    num_points = int(len_sim * pps) + 1

    ### First set the time variables
    t_solve_ODE = np.linspace(0, len_sim, num_points) # timepoints at which the ODE is solved
    t_stim = t_solve_ODE # timepoints at which the stimulus is defined

    inds = (np.arange(len(t_solve_ODE)) % 10 == 0) & (t_solve_ODE <= time[-1])
    t_out = t_solve_ODE[inds] # timepoints at which the output is saved
    assert np.all(np.isclose(t_out, time))

    # First parameters of the integrators have to be set
    integrator_object = icomo.ODEIntegrator(
        ts_out=t_out,
        t_0=min(t_solve_ODE),
        ts_solver=t_solve_ODE,
        ts_arg=t_stim,
        max_steps=len(t_solve_ODE),
    )
    return integrator_object, t_stim

def get_lognormal_params(mean, std):
    sigma = np.sqrt(np.log(std**2 / mean**2 + 1))
    mu = np.log(mean) - 0.5 * sigma**2
    return mu, sigma

with pm.Model() as model:

    ###### Priors on the model parameters

    # Priors on the model parameters
    precision = 2
    mu, sigma = get_lognormal_params(_MAX_RECOVERY, _MAX_RECOVERY/precision)
    max_recovery = pm.LogNormal("max_recovery", mu=mu, sigma=sigma)
    mu, sigma = get_lognormal_params(_MAX_LOADED, _MAX_LOADED/precision)
    max_loaded = pm.LogNormal("max_loaded", mu=mu, sigma=sigma)
    mu, sigma = get_lognormal_params(_MAX_DOCKED, _MAX_DOCKED/precision)
    max_docked = pm.LogNormal("max_docked", mu=mu, sigma=sigma)
    mu, sigma = get_lognormal_params(_MAX_PRIMED, _MAX_PRIMED/precision)
    max_primed = pm.LogNormal("max_primed", mu=mu, sigma=sigma)
    
    precision = 0.7
    # set all to 1
    mu, sigma = get_lognormal_params(_RATE_DOCK, _RATE_DOCK/precision)
    rate_recover = pm.LogNormal("rate_recover", mu=mu, sigma=sigma)
    mu, sigma = get_lognormal_params(_RATE_DOCK, _RATE_DOCK/precision)
    rate_load = pm.LogNormal("rate_load", mu=mu, sigma=sigma)
    mu, sigma = get_lognormal_params(_RATE_DOCK, _RATE_DOCK/precision)
    rate_dock = pm.LogNormal("rate_dock", mu=mu, sigma=sigma)
    mu, sigma = get_lognormal_params(_RATE_DOCK, _RATE_DOCK/precision)
    rate_prime = pm.LogNormal("rate_prime", mu=mu, sigma=sigma)
    
    release_probability1 = pm.Beta("release_probability1", alpha=1, beta=2)
    release_probability2 = pm.Beta("release_probability2", alpha=1, beta=2)
    # additional factor to scale from num vesicles to calcium data
    observation_factor1 = pm.HalfFlat("observation_factor1")
    error_model1 = pm.HalfCauchy("error_model1", beta=0.1)
    # additional factor to scale from num vesicles to calcium data
    observation_factor2 = pm.HalfFlat("observation_factor2")
    error_model2 = pm.HalfCauchy("error_model2", beta=0.3)

    const_args_var1 = {
        "RATE_RECOVER": rate_recover,
        "RATE_LOAD": rate_load,
        "RATE_DOCK": rate_dock,
        "RATE_PRIME": rate_prime,
        "MAX_RECOVERY": max_recovery,
        "MAX_LOADED": max_loaded,
        "MAX_DOCKED": max_docked,
        "MAX_PRIMED": max_primed,
        "RELEASE_PROBABILITY": release_probability1,
    }

    const_args_var2 = {
        "RATE_RECOVER": rate_recover,
        "RATE_LOAD": rate_load,
        "RATE_DOCK": rate_dock,
        "RATE_PRIME": rate_prime,
        "MAX_RECOVERY": max_recovery,
        "MAX_LOADED": max_loaded,
        "MAX_DOCKED": max_docked,
        "MAX_PRIMED": max_primed,
        "RELEASE_PROBABILITY": release_probability2,
    }


    ###### experiment 1

    testtime = 2.0 # s
    depletiontimes = [0.4, 4.0, 40.0]
    pausetimes = [1.0, 10.0, 40.0, 100.0]
    stim_rate = 20.0 # Hz
    conditions = [(depletiontime, pausetime) for pausetime in pausetimes for depletiontime in depletiontimes]
    conditions.append((40.0, 200.0))

    sim = []
    sim_error = []
    obs = []

    for condition in conditions:
        depletiontime, pausetime = condition
        # get condition name
        fm = lambda x: round(x, 1) if x % 1 else int(x)
        condition_name = "{}_{}".format(fm(depletiontime), fm(pausetime))

        y0 = {"recovery": max_recovery, "loaded": max_loaded, "docked": max_docked, "primed": max_primed, "fused": 0.0}
        
        # define integrator object
        integrator_object, ts = get_integrator_object(depletiontime, pausetime, testtime)
        integrator_op = integrator_object.get_op(equations, list_keys_to_return=["primed", "fused"], return_shapes=[() for _ in range(2)])
        
        # define stimulus
        stim = ts < depletiontime
        stim = stim + (ts > depletiontime + pausetime)
        stim = stim * stim_rate

        # And solve the ODE for our starting conditions and parameters
        primed, fused = integrator_op(y0=y0, arg_t=stim, constant_args=const_args_var1)
        # util = calcium + ca_increase * (1 - calcium) 
        release_rate = primed * stim_rate * release_probability1 #*util
        released = pm.math.mean(release_rate)

        # save sim data under condition name
        pm.Deterministic(condition_name, observation_factor1 * released)

        # get data and simulation results
        data1 = df1[condition_name]
        error_of_the_mean = np.std(data1) / np.sqrt(len(data1))
        data1_mean = np.mean(data1)
        sigma_error = pm.Deterministic("scaled_sigma_error_" + condition_name, error_model1 + error_of_the_mean)
        
        sim.append(observation_factor1 * released)
        sim_error.append(sigma_error)
        obs.append(data1_mean)

    sim = pm.math.stack(sim)
    sim_error = pm.math.stack(sim_error)

    mu1 = sim
    sigma1 = sim_error
    observed1 = obs


    ###### experiment 2
    
    stim_rate = 5.0 # Hz
    max_vesicles = _MAX_FUSED + max_loaded + max_docked + max_primed


    ### Define starting conditions
    y0 = {"recovery": max_recovery, "loaded": max_loaded, "docked": max_docked, "primed": max_primed, "fused": 0.0}

    # Get integrator object
    integrator_object, ts = get_integrator_object2()
    integrator = integrator_object.get_op(equations, list_keys_to_return=["primed", "fused"], return_shapes=[() for _ in range(2)])

    ### Set input
    stim = stim_rate * np.ones(len(ts))
    t_args = stim

    # And solve the ODE for our starting conditions and parameters
    primed, _ = integrator(y0=y0, arg_t=t_args, constant_args=const_args_var2)

    release_rate = primed * stim_rate * release_probability2
    pps_experiment = 1.7
    fused = pm.math.cumsum(release_rate) / pps_experiment # have to get total number of vesicles released
    fused -= fused[0]

    out = pm.Deterministic("experiment2", observation_factor2 * fused)
    sim2_errors = pm.Deterministic("scaled_sigma_error_experiment2", error_model2 + data2_errs)

    mu2 = out
    sigma2 = sim2_errors
    observed2 = data2_means

    observed = np.concatenate([observed1, observed2])
    mu = ptt.concatenate([mu1, mu2])
    sigma = ptt.concatenate([sigma1, sigma2])

    def logp(observed, mu, sigma):
        n = 13

        observed1 = observed[:n]
        observed2 = observed[n:]
        mu1 = mu[:n]
        mu2 = mu[n:]
        sigma1 = sigma[:n]
        sigma2 = sigma[n:]
        weight1 = 10
        weight2 = 1
        normalizer = weight1 + weight2
        weight1 /= normalizer / 2
        weight2 /= normalizer / 2
        p1 = pm.logp(pm.Normal.dist(mu1, sigma1),observed1)
        p2 = pm.logp(pm.Normal.dist(mu2, sigma2),observed2)
        return ptt.concatenate([weight1 * p1, weight2 * p2])
         
    pm.CustomDist("likelihood",
                    mu, sigma,
                    logp = logp,
                    observed = observed)
    

In [ ]:
# sample the model
trace = pm.sample(
    model=model,
    tune=800,
    draws=2500,
    cores=4,
    nuts_sampler_kwargs={"nuts_kwargs": {"max_tree_depth": 6}},
    nuts_sampler="numpyro",
    target_accept=0.90,
)

# save the trace
az.to_netcdf(trace, "trace_all_same.nc")


In [ ]:
# # map estimate
# map_estimate = pm.find_MAP(model=model)
# print(map_estimate)

In [ ]:

# # get the posterior
# sim_df = pd.DataFrame()
# for condition in conditions:
#     depletiontime, pausetime = condition
#     condition_name = "{}_{}".format(fm(depletiontime), fm(pausetime))
#     sim_df[condition_name] = np.array([map_estimate[condition_name]])

# fig = plt.figure(figsize=(3.5,2.8))
# ax = fig.subplots(1,1)

# # offset points
# import matplotlib.transforms as transforms
# offset = lambda p: transforms.ScaledTranslation(p/72.,0, fig.dpi_scale_trans)
# trans = ax.transData

# # remove the right and top and bottom spines
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# ax.spines['bottom'].set_visible(False)

# errs_df = df1.apply(lambda x: bootstrap_confidence_interval(x), axis=0)
# plt.errorbar(df1.columns, df1.mean(), yerr=errs_df, linestyle='None', marker='None', markersize=2, elinewidth=9, color='black', alpha=0.2)
# plt.errorbar(df1.columns, df1.mean(), yerr=errs_df, linestyle='None', marker='_', markersize=9, elinewidth=0, color='black', alpha=0.2)

# errs_sim_df = sim_df.apply(lambda x: percentile_confidence_interval(x), axis=0)
# plt.plot(sim_df.columns, sim_df.mean(), linestyle='None', marker='o', color='cornflowerblue')

# plt.xticks(rotation=45)
# plt.ylabel("change in dF/F")
# plt.tight_layout()
# plt.savefig("posterior_full_comparison.pdf", bbox_inches='tight')

In [ ]:
# # get the posteriors
# simulation_results = np.array([map_estimate["experiment2"]])

# simulation_results_mean = np.mean(simulation_results, axis=0)
# # simulation_results_conf = np.percentile(simulation_results, [2.5, 97.5], axis=0)

# deviation = np.std(data2, axis=0) / np.sqrt(data2.shape[0])
# experiment_conf = data2_means + 1.96 * np.array([-deviation, deviation])

# fig, axs = plt.subplots(1,1, figsize=(2.5,2))
# axs.plot(time, data2_means, label="Experimental data", alpha=0.2, color='black')
# axs.fill_between(time, experiment_conf[0], experiment_conf[1], alpha=0.2, color='black')
# axs.plot(time, simulation_results_mean, label="Model prediction", color='cornflowerblue')
# # axs.fill_between(time, simulation_results_conf[0], simulation_results_conf[1], alpha=0.2, color='cornflowerblue')
# axs.set_xlabel("Time [s]")
# axs.set_ylabel("Flourescence increase")
# axs.spines['top'].set_visible(False)
# axs.spines['right'].set_visible(False)
# axs.legend()


# plt.tight_layout() 
# plt.savefig("exhaustion_comparison.pdf", bbox_inches='tight')
# plt.show()

In [ ]:
# load the trace
trace = az.from_netcdf("trace_all_same.nc")
# if chains do not converge throw them out
# trace_full = trace
# trace = trace_full.sel(chain=[0,1,2])

In [ ]:
print(az.rhat(trace))

In [ ]:
# get the posterior
sim_df = pd.DataFrame()
for condition in conditions:
    depletiontime, pausetime = condition
    condition_name = "{}_{}".format(fm(depletiontime), fm(pausetime))
    sim_df[condition_name] = trace.posterior[condition_name].to_numpy().flatten()

fig = plt.figure(figsize=(3.5,2.8))
ax = fig.subplots(1,1)

# offset points
import matplotlib.transforms as transforms
offset = lambda p: transforms.ScaledTranslation(p/72.,0, fig.dpi_scale_trans)
trans = ax.transData

# remove the right and top and bottom spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)

errs_df = df1.apply(lambda x: bootstrap_confidence_interval(x), axis=0)
plt.errorbar(df1.columns, df1.mean(), yerr=errs_df, linestyle='None', marker='None', markersize=2, elinewidth=9, color='black', alpha=0.2)
plt.errorbar(df1.columns, df1.mean(), yerr=errs_df, linestyle='None', marker='_', markersize=9, elinewidth=0, color='black', alpha=0.2)

errs_sim_df = sim_df.apply(lambda x: percentile_confidence_interval(x), axis=0)
plt.errorbar(sim_df.columns, sim_df.mean(), yerr=errs_sim_df, linestyle='None', marker='o', color='cornflowerblue')

plt.xticks(rotation=45)
plt.ylabel("change in dF/F")
plt.tight_layout()
plt.savefig("posterior_all_same_exp1.pdf", bbox_inches='tight')


In [ ]:
# get the posteriors
simulation_results = trace.posterior["experiment2"].to_numpy().reshape(-1, data2.shape[1])

simulation_results_mean = np.mean(simulation_results, axis=0)
simulation_results_conf = np.percentile(simulation_results, [2.5, 97.5], axis=0)

deviation = np.std(data2, axis=0) / np.sqrt(data2.shape[0])
experiment_conf = data2_means + 1.96 * np.array([-deviation, deviation])

fig, axs = plt.subplots(1,1, figsize=(2.5,2))
axs.plot(time, data2_means, label="Experimental data", alpha=0.2, color='black')
axs.fill_between(time, experiment_conf[0], experiment_conf[1], alpha=0.2, color='black')
axs.plot(time, simulation_results_mean, label="Model prediction", color='cornflowerblue')
axs.fill_between(time, simulation_results_conf[0], simulation_results_conf[1], alpha=0.2, color='cornflowerblue')
axs.set_xlabel("Time [s]")
axs.set_ylabel("Flourescence increase")
axs.spines['top'].set_visible(False)
axs.spines['right'].set_visible(False)
# axs.legend()


plt.tight_layout() 
plt.savefig("posterior_all_same_exp2.pdf", bbox_inches='tight')
plt.show()

In [ ]:
def plot_cont(prior, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    samples = pm.draw(prior, draws=1000)
    x = np.linspace(np.min(samples), np.max(samples), 1000)
    ax.plot(x, np.exp(pm.logp(prior,x)).eval(), color='gray')
    return ax

condition_names = ["{}_{}".format(fm(condition[0]), fm(condition[1])) for condition in conditions]

name_dict = {
    "max_recovery": r"$P^{max}_{recovery}$",
    "max_loaded": r"$P^{max}_{loaded}$",
    "max_docked": r"$P^{max}_{docked}$",
    "max_primed": r"$P^{max}_{primed}$",
    "rate_recover": r"$r_{fused,recovery}$",
    "rate_load": r"$r_{recovery,loaded}$",
    "rate_dock": r"$r_{loaded,docked}$",
    "rate_prime": r"$r_{docked,primed}$",
    "release_probability1": r"$p^{(1)}_{fuse}$",
    "release_probability2": r"$p^{(2)}_{fuse}$",
    "observation_factor1": r"$\alpha_{obs}^{(1)}$",
    "observation_factor2": r"$\alpha_{obs}^{(2)}$",
    "error_model1": r"$\sigma_{obs}^{(1)}$",
    "error_model2": r"$\sigma_{obs}^{(2)}$",
    "rate_ca": r"$r_{ca}$",
    "ca_increase": r"$\Delta_{ca}$",
}

# for each variable plot prior and posterior
names = [var.name for var in model.unobserved_RVs]
fl = lambda name: not any([condition_name in name for condition_name in condition_names]) and not "experiment2" in name
names = list(filter(fl, names))

sidelength = int(np.ceil(np.sqrt(len(names))))
fig, axs = plt.subplots(sidelength, sidelength, figsize=(sidelength*2.5, sidelength*2));
axs = axs.flatten()

for name, i in zip(names, range(len(names))):
    ax = axs[i]

    pm.plot_posterior(trace, var_names=name, color='cornflowerblue', ax=ax)
    ax.set_title(name_dict[name], fontdict={'fontsize': 15})
    # plot prior
    prior = model[name]
    try:
        plot_cont(prior, ax=ax)
    except:
        pass
    # scale to posterior
    if name == "error_model":
        ax.set_xlim((0, np.max(trace.posterior[name])))

plt.tight_layout();
plt.savefig("posterior_all_same.pdf", bbox_inches='tight')



In [ ]:
print(az.summary(trace))
# keys = list(const_args_var.keys())
# keys = [k.lower() for k in keys]
# az.plot_forest(trace, var_names=keys, combined=True, hdi_prob=0.95, transform=lambda x: np.log10(x))
# az.plot_forest(trace, var_names=["error_model"], combined=True, hdi_prob=0.95)